## Importing and getting to know the Df

In [1]:
import os  # operating system interface 
print(os.getcwd())

c:\Users\Diya\Documents\GItHub\DELTA_Element3


In [6]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
#with gzip.open('btc_tweets_train.parquet.gzip', 'rt') as f:
 #   content = f.read()
    # Process the content

In [7]:
df = pd.read_parquet('datasets/btc_tweets_train.parquet.gzip', engine='pyarrow')
# Display the first few rows of the dataframe
df.head()

,hashtags,content,username,user_displayname,sentiment
tweet ID,,,,,
1641579121972236290,"[Bitcoin, Bitcoin, BTC, Bitcoin, BTC, SHIB, HO...","$Bitcoin TO $100,000 SOONER THAN YOU THINK‼️💯🙏...",BezosCrypto,SHIB Bezos,True
1641579176171016194,"[Bitcoin, bitcoinordinals, crypto]",Alright I have my rares. Who else is grabbing ...,spartantc81,SpartanTC,True
1641579486071390208,"[BTC, SHIB, HOGE, SAITAMA, BNB, DOGE, ETH, Bab...","Bitcoin (BTC) Targets Over $100,000 as This Im...",BezosCrypto,SHIB Bezos,True
1641579537103302656,[BTC],📢 Xverse Web-based pool is live:\n\n•Update @x...,godfred_xcuz,Algorithm.btc,True
1641579588399804418,[Bitcoin],"Yesterday, a Bitcoin projection was displayed ...",goddess81oo,she is lucky,True


In [ ]:
df.info()

In [ ]:
df.isnull().sum().sum()

In [ ]:
df.iloc[1299,1]

In [ ]:
df['sentiment'].value_counts()

## Df Cleaning

In [ ]:
df.reset_index(inplace=True)
df.drop(columns=['tweet ID','hashtags','username','user_displayname'], inplace=True)

In [ ]:
#Get the number of duplicated rows and remove any duplicates after that:
print('Number of duplicated samples: ',df[df.duplicated()].shape[0])
df=df.drop_duplicates(keep='first')

In [ ]:
df.head()

## Text Cleaning

In [ ]:
def remove_whitespace(text):
    """ Function to remove whitespace (tabs, newlines). """
    return ' '.join(text.split())

In [ ]:
import re
def remove_links(text):
    """Function to remove hyperlinks from the text."""
    return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

In [ ]:
def remove_punctuation_and_casing(text):
    """
    Function to remove the punctuation, upper casing and words that include
    non-alphanumeric characters.
    """
    chars = '!\"#$%&()*+,-./:;<=>?@[\]^_`{|}~'
    text = text.translate(str.maketrans(chars, ' ' * len(chars)))
    return ' '.join([word.lower() for word in text.split() if word.isalpha()])

In [ ]:
from nltk.corpus import stopwords

english_stopwords = stopwords.words('english')

def remove_stopwords(text):
    """ Function to remove stopwords. """
    return ' '.join([word for word in str(text).split() if not word in english_stopwords])

In [ ]:
# NLTK lemmatization
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_words(text, **kwargs):
    """ Function to lemmatize words. """
    return ' '.join([lemmatizer.lemmatize(word, **kwargs) for word in text.split()])

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
def preprocess_text(text):
    """Function to preprocess text by removing links, whitespace, punctuation, and converting to lower case."""
    text = remove_links(text)
    text = remove_whitespace(text)
    text = remove_punctuation_and_casing(text)
    text = remove_stopwords(text)
    text = lemmatize_words(text)
    text =  ' '.join(word_tokenize(text)) #to return only the text and not a list
#    text = stem_words(text)
    return text

In [ ]:
df['cleaned_content'] = df['content'].apply(preprocess_text)

In [ ]:
df.head()

In [ ]:
## After the process:
# Check all is well
ix = 12  # just one example, play with other play to further examine the effect of our clearning
print('Original Review:\n' + df.content[ix])  
print('\nCleaned Review:\n' + df.cleaned_content[ix])
#from nlp_foundations

## (1) Sentiment Dictionary Benchmark - vaderSentiment

In [ ]:
#pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re

def remove_links(text):
    return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

def remove_whitespace(text):
    return ' '.join(text.split())

def remove_punctuation_and_casing(text):
    chars = '!\"#$%&()*+,-./:;<=>?@[\]^_`{|}~'
    text = text.translate(str.maketrans(chars, ' ' * len(chars)))
    return ' '.join([word.lower() for word in text.split() if word.isalpha()])

def preprocess_tweet(text):
    text = remove_links(text)
    text = remove_whitespace(text)
    text = remove_punctuation_and_casing(text)
    return text

# Example tweets
tweets = ["I love this product! #awesome", "This is the worst experience ever! http://example.com"]

# Preprocess tweets
preprocessed_tweets = [preprocess_tweet(tweet) for tweet in tweets]

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Analyze sentiment
for tweet in preprocessed_tweets:
    sentiment = analyzer.polarity_scores(tweet)
    print(f"Tweet: {tweet}")
    print(f"Sentiment: {sentiment}")
    print("\n")

In [ ]:
#dictionary = pd.DataFrame(df['cleaned_content'])

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    sentiment_dict = analyzer.polarity_scores(text)
    return sentiment_dict

# Apply sentiment analysis to each tweet
df['vader_sentiment'] = df['cleaned_content'].apply(analyze_sentiment)

# Extract compound score for simplicity
df['vader_compound'] = df['vader_sentiment'].apply(lambda x: x['compound'])

In [ ]:
print(df['vader_compound'].head())

In [ ]:
pd.set_option('display.max_colwidth', None)
print(df['vader_sentiment'].head())

In [ ]:
# Convert compound score to sentiment label
def compound_to_binary_sentiment(compound_score):
    if compound_score > 0:
        return 1  # Positive
    elif compound_score <= 0:
        return 0  # Negative

df['vader_label'] = df['vader_compound'].apply(compound_to_binary_sentiment)

### Sentiment classifier assessment

In [ ]:
#Comparison (If you have actual labels, you can evaluate the performance)
if 'sentiment' in df.columns:
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

    # Calculate metrics
    accuracy = accuracy_score(df['sentiment'], df['vader_label'])
    precision = precision_score(df['sentiment'], df['vader_label'], average='weighted')
    recall = recall_score(df['sentiment'], df['vader_label'], average='weighted')
    f1 = f1_score(df['sentiment'], df['vader_label'], average='weighted')

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}")

In [ ]:
df.head()

In [ ]:
df['vader_compound'].hist(); 
df['vader_compound'].describe() # overall rather positive

In [ ]:
#check #https://github.com/Humboldt-WI/delta/blob/master/demos/nlp/sentiment_analysis.ipynb

## Check dropping neutral compound values

In [ ]:
def compound_to_binary_sentiment_2(compound_score):
    if compound_score >= 0.05:
        return 1  # Positive
    elif compound_score <= -0.05:
        return 0  # Negative
    else:
        return None  # Neutral, will be ignored

df['vader_binary_label_2'] = df['vader_compound'].apply(compound_to_binary_sentiment_2)

In [ ]:
temp = df.copy()
temp = temp.dropna()

In [ ]:
temp['vader_binary_label_2'].isna().sum()

In [ ]:
# Convert boolean values to integers
temp['sentiment'] = temp['sentiment'].astype(int)

# Print the first few rows to verify the change
print(temp.head())

In [ ]:
#Comparison (If you have actual labels, you can evaluate the performance)
if 'sentiment' in temp.columns:
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

    # Calculate metrics
    accuracy = accuracy_score(temp['sentiment'], temp['vader_binary_label_2'])
    precision = precision_score(temp['sentiment'], temp['vader_binary_label_2'], average='weighted')
    recall = recall_score(temp['sentiment'], temp['vader_binary_label_2'], average='weighted')
    f1 = f1_score(temp['sentiment'], temp['vader_binary_label_2'], average='weighted')

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}")

In [ ]:
# Print the DataFrame to see the results
df.head()

## Embedding - FastText

In [ ]:
from gensim.models import Word2Vec
import nltk
from nltk.corpus import brown

# Download and load the sample dataset
nltk.download('brown')
sentences = brown.sents()

# Train the Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)

# Save the model
model.save("word2vec.model")

# Load the model
model = Word2Vec.load("word2vec.model")

# Get the vector for a word
vector = model.wv['king']
print(vector)

# Find most similar words
similar_words = model.wv.most_similar('king')
print(similar_words)


In [ ]:
from gensim.models import FastText

In [ ]:
type(df['cleaned_content'].iloc[1,])

In [ ]:
df['cleaned_content'].iloc[1,]

In [ ]:
# Tokenize the cleaned tweets (split by spaces)
tokenized_tweets = [tweet.split() for tweet in df['cleaned_content']]

# Train FastText model using Gensim's implementation
fasttext_model = FastText(sentences=tokenized_tweets, vector_size=100, window=5, min_count=3, sg=1, epochs=10)

# Save the model
fasttext_model.save("fasttext.model")

# Load the model (for future use)
fasttext_model = FastText.load("fasttext.model")

# Example: Get vector for a word 
print(f"Vector for 'bitcoin': {fasttext_model.wv['bitcoin']}")

# Example: Get most similar words
print(f"Words similar to 'bitcoin': {fasttext_model.wv.most_similar('bitcoin')}")

In [ ]:
type(df)

## (2) RNN-based language classifier

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, LSTM, Dense, Dropout, SimpleRNN
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

Notes: 
- So using two embedding layers is important.


### Partitioning

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df['cleaned_content'],df['sentiment'], test_size = 0.2, random_state = 5)

In [ ]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
type(X_train)

### Building vocabulary - Tokenizing & Padding

In [ ]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Tokenize the text
tokenizer = Tokenizer(num_words=5000, oov_token=1)
# The oov_token is a placeholder token that replaces any OOV words during the text_to_sequence calls.
#This ensures that your model can handle new, unseen words gracefully.
tokenizer.fit_on_texts(X_train)

In [ ]:
# On how many tweets did we train?
print(tokenizer.document_count)

In [ ]:
# How many unique words?
len(tokenizer.word_counts)

In [ ]:
# A dictionary of words storing in how many documents a word appeared
word = 'bitcoin'  
n = tokenizer.word_docs[word]
print('The word <{}> appeared in {} tweets.'.format(word, n))

In [ ]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

In [ ]:
import matplotlib.pyplot as plt
# Calculate the number of tokens per document
num_tokens_per_doc = [len(seq) for seq in X_train_seq]

# Plot the distribution
plt.hist(num_tokens_per_doc, bins=50)
plt.xlabel('Number of Tokens')
plt.ylabel('Number of Documents')
plt.title('Distribution of Tokens per Document')
plt.show()

In [ ]:
import numpy as np

# Calculate the 95th percentile of the number of tokens per document
max_length = int(np.percentile(num_tokens_per_doc, 95))
print(f"Recommended max length for padding: {max_length}")


In [ ]:
# Pad the sequences
max_length = 28  # Set max length for padding
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=max_length, padding='post')

### Load own (FastText) Embeddings

In [ ]:
# Create an embedding matrix
embedding_dim = 100  # Must match the dimension of FastText vectors
vocab_size = len(tokenizer.word_index) + 1   #it's the total number of unique words in your tokenizer’s vocabulary plus one. The +1 accounts for the padding token (index 0).
embedding_matrix = np.zeros((vocab_size, embedding_dim)) #embedding_matrix is initialized as a matrix of zeros with shape (vocab_size, embedding_dim). 
                                                        #This matrix will eventually hold the FastText vectors for each word in your vocabulary.

for word, i in tokenizer.word_index.items():
    if word in fasttext_model.wv:
        embedding_matrix[i] = fasttext_model.wv[word]

print(f"Embedding matrix shape: {embedding_matrix.shape}")

In [ ]:
embedding_matrix

### Simple RNN Model

In [ ]:
# Model definition
SR_model = Sequential() # Input shape
SR_model.add(SimpleRNN(100, activation='relu', input_shape=(5, 321)))  # First RNN layer with 50 units
SR_model.add(Dense(1))  # Output layer
SR_model.summary()

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
#Train the model
history = model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_data=(X_val_pad, y_val))

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_val_pad, y_val)
print(f"Test Accuracy: {accuracy}")

# Predict on test data
y_pred = model.predict(X_val_pad)
y_pred = (y_pred > 0.5).astype("int32")

from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred))


###False (Class 0):

Precision: 1.00
All predictions made for the "False" class are correct. However, given the other metrics, this likely means very few predictions were made as "False."
Recall: 0.02
The model only correctly identified 2% of the actual "False" instances. This indicates that the model is largely missing instances of this class.
F1-Score: 0.04
The low F1-score reflects the imbalance between the high precision and very low recall.
Support: 55
There are 55 instances of the "False" class in the dataset.


###True (Class 1):
Precision: 0.82
82% of the predictions for the "True" class are correct.
Recall: 1.00
The model correctly identifies 100% of the "True" instances. However, this could mean that the model is over-predicting this class.
F1-Score: 0.90
The F1-score is high due to the perfect recall and reasonably good precision.
Support: 245
There are 245 instances of the "True" class in the dataset.
Accuracy: 0.82

The model correctly predicted 82% of the total instances in the dataset.

###Macro Average:
Precision: 0.91
Average precision across both classes.
Recall: 0.51
Average recall across both classes. The low value here reflects the poor performance on the "False" class.
F1-Score: 0.47
Average F1-score across both classes, showing an imbalance due to the very low performance on the "False" class.

###Weighted Average:
Precision: 0.85
Recall: 0.82
F1-Score: 0.74
These metrics are weighted by the support (number of instances) for each class, showing a more balanced view but still reflecting the issues with class imbalance.

Interpretation of the Results
Class Imbalance: The model seems to have a significant class imbalance issue, where it overwhelmingly predicts the "True" class (Class 1) and fails to correctly identify the "False" class (Class 0).

High Precision for False but Low Recall: The model is very conservative when predicting the "False" class (high precision), but it misses most of the actual "False" instances (low recall).

Potential Overfitting to True Class: The model might be overfitting to the "True" class, leading to high recall and reasonable precision but at the cost of completely neglecting the "False" class.

## Extra Stuff - To review  

In [ ]:
import pysentiment2 as ps
dc = ps.HIV4() # import Harvard IV-4 dictionary

def get_sentiment_score(text):
    score = round(dc.get_score(dc.tokenize(text))['Polarity'], 2)
    return score

In [ ]:
# Do the cleaning
# CAUTION: depending on your data set size, the processing might take a while 
import time  # To keep an eye on runtimes
start = time.time()
imdb_data['review_clean'] = text_cleaning(imdb_data.review)
print('Duration: {:.0f} sec'.format(time.time()-start))